In [25]:
import shutil
import os
import numpy as np
import keras
from keras.applications import VGG16
from keras import models
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
import matplotlib as mplt
import re

In [2]:
directory = r'/home/josheen/projects/def-vinay/josheen/DL/Figure13model/runs_excel_files'
dfs = []
for runs in os.listdir(directory):
    R = pd.read_excel(os.path.join(directory,runs))
    rnum = os.path.splitext(runs)[0]
    R.head()
    R['Run'] = int(rnum[1])
    R.head()
    dfs.append(R)
data = pd.concat(dfs)

In [3]:
FileName = "/home/josheen/projects/def-vinay/josheen/DL/Figure13model/mapped_data/AllRunsmapped.csv"
directory = "/home/josheen/projects/def-vinay/josheen/DL/"
df = pd.read_csv(FileName)
for index, row in df.iterrows():
    address = row['Address']
    new = address.replace("\\", "/")
    add_index = address.find('Figure13model')
    new_filename = os.path.join(directory, new[add_index:])
    df.at[index, 'Address'] = new_filename
    
df.to_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv', index = False, header=True)

In [23]:
df = pd.read_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv')
df = df[df.Element == 'Lead']
left=115
top=35
right=243
bottom=163

train_files = []
y_train = []
dataset = []
for index, row in df.iterrows():
    y_train.append(row['Recovery']) 
    img = load_img(row['Address'])  # this is a PIL image
    
    img1 = img.crop((left,top,right,bottom))
    
    #img2 =img1.thumbnail((128, 128))
    x = img_to_array(img1)
    #x = x.reshape((3, 128, 128))
    #x = (x - 128.0) / 128.0
    x = (x - 255.0) / 255.0
    dataset.append(x)

In [9]:
from sklearn.model_selection import train_test_split

test_to_train=0.4
val_to_test=0.5

X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=test_to_train, random_state=33)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_to_test, random_state=33)
print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))

Train set size: 613, Val set size: 205, Test set size: 204


# Pretrained CNN - VGG16

notes: we should look into making generators, accuracy does not seem to go higher than MAE = 1.5.

In [10]:
conv_base = VGG16(weights = 'imagenet', include_top=False, input_shape=(128,128,3))
conv_base.trainable = False 
CNN = models.Sequential()
CNN.add(conv_base)
CNN.add(layers.Flatten())
CNN.add(layers.Dense(256, activation='relu'))
CNN.add(layers.Dropout(0.2))
CNN.add(layers.Dense(32, activation='relu'))
CNN.add(layers.Dropout(0.3))
CNN.add(layers.Dense(1, activation = 'linear'))
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                

In [11]:
CNN.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
my_callbacks = EarlyStopping(monitor="mean_absolute_error", patience=6, mode="auto")
X_val = np.array(X_val)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_val = np.array(y_val)
CNN.fit(X_train, y_train, batch_size=32,epochs=75,validation_data=(X_val, y_val))
model = CNN.save('/home/josheen/projects/def-vinay/josheen/DL/')


Epoch 1/75
20/20 [==============================] - 32s 2s/step - loss: 27.9036 - mae: 4.1165 - val_loss: 4.7617 - val_mae: 1.6448
Epoch 2/75
20/20 [==============================] - 32s 2s/step - loss: 12.5647 - mae: 2.8332 - val_loss: 2.8762 - val_mae: 1.2607
Epoch 3/75
20/20 [==============================] - 33s 2s/step - loss: 11.3813 - mae: 2.6085 - val_loss: 3.0765 - val_mae: 1.2197
Epoch 4/75
20/20 [==============================] - 33s 2s/step - loss: 9.7415 - mae: 2.4728 - val_loss: 11.8371 - val_mae: 3.1843
Epoch 5/75
20/20 [==============================] - 33s 2s/step - loss: 11.1783 - mae: 2.6532 - val_loss: 2.5852 - val_mae: 1.2561
Epoch 6/75
20/20 [==============================] - 33s 2s/step - loss: 9.2012 - mae: 2.4312 - val_loss: 4.7706 - val_mae: 1.8183
Epoch 7/75
20/20 [==============================] - 33s 2s/step - loss: 8.0253 - mae: 2.2152 - val_loss: 1.8967 - val_mae: 1.0165
Epoch 8/75
20/20 [==============================] - 33s 2s/step - loss: 9.8062 - mae:

# GAN

In [15]:
print('maximum recovery is {}'.format(df['Recovery'].max())) 
print('min recovery is {}'.format(df['Recovery'].min()))
df.sort_values('Recovery')
df 

maximum recovery is 19.1278815972938
min recovery is 1.938791471502


,Address,Element,FileName,Recovery,Run,SheetName,Sl_No,Stage
0,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 19h 06m 48.7750s.jpg,7.548923,1,R1_Lead,2,1
1,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 19h 06m 58.5150s.jpg,8.758845,1,R1_Lead,3,1
2,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 19h 07m 10.1370s.jpg,9.605029,1,R1_Lead,4,1
3,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 19h 07m 20.7539s.jpg,10.123912,1,R1_Lead,5,1
4,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 19h 07m 32.409s.jpg,10.577631,1,R1_Lead,6,1
...,...,...,...,...,...,...,...,...
2365,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 22h 14m 25.7990s.jpg,13.122611,9,R9_Lead,2953,1
2366,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 22h 14m 27.7219s.jpg,13.208679,9,R9_Lead,2954,1
2367,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 22h 14m 29.9579s.jpg,13.324618,9,R9_Lead,2955,1
2368,/home/josheen/projects/def-vinay/josheen/DL/Fi...,Lead,Cell1 22h 14m 31.8889s.jpg,13.339733,9,R9_Lead,2956,1


In [28]:
#consider splitting from 0-3 3-6 6-9 9-12 12-15 15-18 18-21 7 classes
categorical_train = []
dataset = []

num_classes = 7
for index, row in df.iterrows():
    if row['Recovery'] <= 3:
        categorical_train.append(0)
    elif row['Recovery'] > 3 and row['Recovery'] <= 6:
        categorical_train.append(1)
    elif row['Recovery'] > 6 and row['Recovery']<= 9:
        categorical_train.append(2)
    elif row['Recovery'] > 9 and row['Recovery']<= 12:
        categorical_train.append(3)
    elif row['Recovery'] > 12 and row['Recovery']<= 15:
        categorical_train.append(4)
    elif row['Recovery'] > 15 and row['Recovery']<= 18:
        categorical_train.append(5)
    elif row['Recovery'] > 18:
        categorical_train.append(6)
categorical_train = keras.utils.to_categorical(categorical_train, 7)

In [35]:
#discriminator
input_shape = (128,128,3)
input_im = Input(shape = input_shape)
fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(input_im)
fe = LeakyReLU(alpha=0.2)(fe)
fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
fe = LeakyReLU(alpha=0.2)(fe)
fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
fe = LeakyReLU(alpha=0.2)(fe)
fe = Flatten()(fe)
fe = Dropout(0.4)(fe)
d_out_layer = Dense(7, activation='sigmoid')(fe)
discriminator = keras.models.Model(input_im, d_out_layer)
discriminator.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


def define_gan(g_model, d_model):
    d_model.trainable = false
    gan_output = d_model(g_model.output)
    model = model(g_model.input, gan_output)
    loss = Adam()

In [ ]:
#generator
latent_dim = 256
in_lat = Input(shape=(latent_dim,))
# foundation for 7x7 image
n_nodes = 128 * 128 * 128 * 3
gen = Dense(n_nodes)(in_lat)
gen = LeakyReLU(alpha=0.2)(gen)
gen = Reshape((7, 7, 128))(gen)
# upsample to 14x14
gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
gen = LeakyReLU(alpha=0.2)(gen)
# upsample to 28x28
gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
gen = LeakyReLU(alpha=0.2)(gen)

gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
gen = LeakyReLU(alpha=0.2)(gen)
out_layer = Conv2D(1, (128,128,3), activation='tanh', padding='same')(gen)
# define model
generator = Model(in_lat, out_layer)
